<a href="https://colab.research.google.com/github/DimaKav/DS-Unit-2-Sprint-4-Model-Validation/blob/master/DS_Unit_2_Sprint_Challenge_4_Model_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [0]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [0]:
df['made_donation_in_march_2007'].value_counts()

0    570
1    178
Name: made_donation_in_march_2007, dtype: int64

0 is our majority class

In [0]:
X = df.drop('made_donation_in_march_2007',axis=1)
y = df['made_donation_in_march_2007']
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y)

majority_class = 0
y_pred = [majority_class] * len(y_val)

from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.7754010695187166

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [0]:
from sklearn.metrics import recall_score
recall_score(y_val, y_pred)

0.0

## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75, random_state=42)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [0]:
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

pipe = make_pipeline(
    RobustScaler(), 
    SelectKBest(f_classif), 
    LogisticRegression(solver='lbfgs', random_state=42))

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [0]:
param_grid = {
    'selectkbest__k': [1,2,3,4], 
    'logisticregression__class_weight' : [None, 'balanced'],
    'logisticregression__C' : [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0]
}

# Grid search
gs = GridSearchCV(pipe, param_grid=param_grid, cv=5, 
                  scoring='neg_mean_absolute_error')

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x1a1a8726a8>)), ('logisticregression', LogisticRegression(C=1.0, cla...penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'selectkbest__k': [1, 2, 3, 4], 'logisticregression__class_weight': [None, 'balanced'], 'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [0]:
print('Best params:', gs.best_params_)
print('Best score:', gs.best_score_)

Best params: {'logisticregression__C': 1.0, 'logisticregression__class_weight': None, 'selectkbest__k': 4}
Best score: -0.2192513368983957


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

In [0]:
true_negative  = 85
false_positive = 58
false_negative = 8
true_positive  = 36

accuracy = ((true_negative + true_positive) / 
            (true_negative + false_positive + false_negative + true_positive))
accuracy

0.6470588235294118

Calculate precision

In [0]:
actual_negative = true_negative + false_positive
actual_positive = false_positive + true_positive

predicted_negative = true_negative + false_negative
predicted_positive = false_positive + true_positive

precision = true_positive / predicted_positive
precision

0.3829787234042553

Calculate recall

In [0]:
recall = true_positive / actual_positive
recall

0.3829787234042553

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 

In [0]:
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [0]:
df.var()

months_since_last_donation     6.553543e+01
number_of_donations            3.409751e+01
total_volume_donated           2.131094e+06
months_since_first_donation    5.942242e+02
made_donation_in_march_2007    1.815819e-01
dtype: float64

In [0]:
# Part 1

# Lets log-normalize the column with high variance
df['total_volume_normed'] = np.log(df['total_volume_donated'])
del df['total_volume_donated']

In [0]:
# Part 2

X = df.drop('made_donation_in_march_2007',axis=1)
y = df['made_donation_in_march_2007']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75, random_state=42)

param_grid = {
    'selectkbest__k': [1,2,3,4], 
    'logisticregression__class_weight' : [None, 'balanced'],
    'logisticregression__C' : [.0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0],
    'logisticregression__intercept_scaling' : [0.001, 0.005, 0.01, 1]
}

# Grid search
gs = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')

gs.fit(X_train, y_train)
print('Train score:', gs.best_score_)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train score: -0.20320855614973263


Score got closer to 0, but it's mostly due to log transformation of high variance variable from Part 1...

In [0]:
# Part 3

# Show names of selected features
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)
    
# Final evaluation on the test set (test score)
test_score = gs.score(X_test, y_test)
print('\nTest Score:', test_score)

Features selected:
months_since_last_donation
number_of_donations
months_since_first_donation
total_volume_normed

Features not selected:

Test Score: -0.24064171122994651


In [0]:
y_train.shape

(561,)

In [0]:
# Part 4

gs = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring='f1')

gs.score(X_train, y_train)


# from sklearn.metrics import roc_auc_score, roc_curve
# fpr, tpr, thresholds = roc_curve(y_train, y_pred_)
# print('False positive rate:', fpr)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.